In [2]:
import pandas as pd
import glob
import math
import xlrd
import openpyxl
import re

# Define date interval quartals

def get_quartal(date):
    if date.month in [1, 2, 3]:
        return 'Q1'
    elif date.month in [4, 5, 6]:
        return 'Q2'
    elif date.month in [7, 8, 9]:
        return 'Q3'
    else:
        return 'Q4'

In [3]:
filepath = "/Users/graciaandriamiadana/Documents/Research Culture PGR project/data/Combined PGR studenship payments 2021-22.xlsx"
df = pd.read_excel(filepath, engine= 'openpyxl')
df.head(5)

,Student ID,Budget code,A/c Code,2021-10-01 00:00:00,2021-11-01 00:00:00,2021-12-01 00:00:00,2022-01-01 00:00:00,2022-02-01 00:00:00,2022-03-01 00:00:00,2022-04-01 00:00:00,...,2022-06-01 00:00:00,2022-07-01 00:00:00,2022-08-01 00:00:00,2022-09-01 00:00:00,School,Start date,End date,Duration,Source,Unnamed: 20
0,200346331,BCCG1D3R,3620,5250,NaN,NaN,5250,NaN,NaN,5250,...,NaN,5250,NaN,NaN,BCI,2021-10-01 00:00:00,2024-09-30 00:00:00,36,CRUK + CoL Centre,NaN
1,130490223,TBYG1V3R,3620,3333.333333,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,BCI,2018-06-01 00:00:00,2021-11-30 00:00:00,36 months + 6,Barts Charity,NaN
2,130234410,CMB6001F,3620,NaN,1877.5,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,BCI,2021-11-01 00:00:00,2021-11-30 00:00:00,1,BCI,NaN
3,200974709,PRN1040R,3620,4402.25,NaN,NaN,4402.25,NaN,NaN,4402.25,...,NaN,4402.25,NaN,NaN,BCI,2020-10-01 00:00:00,2024-09-30 00:00:00,48,BBSRC LIDo,NaN
4,170735292,PRN1040R,3620,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,BCI,2017-10-01 00:00:00,2021-09-30 00:00:00,48 months,MRC,NaN


In [4]:
df = df.loc[:, ~df.columns.astype(str).str.contains('^Unnamed')]

# Reshape the DataFrame using pd.melt() to unpivot date columns
df_melted = pd.melt(df, id_vars=['Student ID', 'Budget code', 'A/c Code', 'School', 'Start date', 'End date', 'Duration', 'Source'],
                    var_name='Date', value_name='Amount')

# Convert 'Date' column to datetime format
df_melted['Date'] = pd.to_datetime(df_melted['Date'], errors='coerce')  # 'coerce' to handle any parsing errors

# # Convert non-numeric strings to NaN in 'Amount' columns
df_melted['Amount'] = pd.to_numeric(df_melted['Amount'], errors='coerce')

# Convert the "Source" column to strings 
df_melted["Source"] = df_melted["Source"].astype(str)

In [17]:
df_melted['School'] = df_melted['School'].fillna('')

# Perform case-insensitive replacements and transformations in the 'School' column of df_melted
# replacements = {
#     r'.*EECS.*': 'School of Electronic Engineering and Computer Science',        
#     r'^.*Geog.*$': 'School of Geography',   
#     r'.*BCI.*': 'Barts Cancer Institute', 
#     r'.*Blizard.*': 'Blizard Institute', 
#     r'.*BLIZARD.*': 'Blizard Institute',  
#     r'^.*SED.*$': 'School of English and Drama' ,          
#     r'^.*IPHS.*$': 'Institute of Population Health Sciences' ,         
#     r'^.*SLLF.*$': 'School of Languages, Linguistics and Film' ,          
#     r'^.*History.*$': 'School of History',           
#     r'^.*Law.*$': 'School of Law',
#     r'^.*WHRI.*$': 'William Harvey Research Institute',
#     r'^.*WIPH.*$': 'Wolfson Institute of Population Health', # Wolfson
#     r'^.*Wolfson.*$': 'Wolfson Institute of Population Health', # Wolfson
#     r'^.*DCE_Maths.*$': 'Data-Centric Engineering/Mathematical Sciences',
#     r'^.*SMS.*$': 'School of Mathematical Sciences',
#     r'^.*IoD.*$': 'Institute of Dentistry',
#     r'^.*SBBS.*$': 'School of Biological and Behavioural Sciences',
#     r'^.*SBM.*$': 'School of Business and Management',
#     r'^.*SEF.*$': 'School of Economics and Finance',
#     r'^.*SEMS.*$': 'School of Engineering and Materials Science',
#     r'^.*SPCS.*$': 'School of Physical and Chemical Sciences',
#     r'^.*SPIR.*$': 'School of Politics and International Relations',
replacements = {
    r'.*EECS.*': 'EECS',        
    # r'^.*Geog.*$': 'School of Geography',   
    r'.*BCI.*': 'BCI', 
    r'.*Blizard.*': 'Blizard', 
    r'.*BLIZARD.*': 'Blizard',  
    r'^.*SBBS.*$': 'SBBS',
    r'^.*SED.*$': 'SED',
    # r'^.*SED.*$': 'School of English and Drama' ,          
    # r'^.*IPHS.*$': 'Institute of Population Health Sciences' ,         
    # r'^.*SLLF.*$': 'School of Languages, Linguistics and Film' ,          
    # r'^.*History.*$': 'School of History',           
    # r'^.*Law.*$': 'School of Law',
    # r'^.*WHRI.*$': 'William Harvey Research Institute',
    r'^.*Wolfson.*$': 'WIPH',
    r'^.*DCE_Maths.*$': 'Maths',
    r'^.*SMS.*$': 'Maths',
    r'^.*IoD.*$': 'Dentistry',
}          


# Iterate over the dictionary and perform replacements
for pattern, replacement in replacements.items():
    mask = df_melted['School'].str.contains(pattern, case=True, regex=True, na=False)
    df_melted.loc[mask, 'School'] = replacement

# Remove rows where 'School' is an empty string
df_melted = df_melted[df_melted['School'] != '']


print(sorted(df_melted["School"].unique().tolist()))

['BCI', 'Blizard', 'Dentistry', 'EECS', 'Geog', 'History', 'IPHS', 'Law', 'Maths', 'SBBS', 'SBM', 'SED', 'SEF', 'SEMS', 'SLLF', 'SPCS', 'SPIR', 'WHRI', 'WIPH']


In [18]:
# Read the Excel file containing the category mappings
mapping_file = '/Users/graciaandriamiadana/Documents/Research Culture PGR project/data/Sources classification.xlsx'
category_mapping = pd.read_excel(mapping_file)

# Initialize an empty dictionary for replacements
replacements = {}

# Iterate over each category and its sub-names to generate regex patterns
for category in category_mapping.columns:
    for name in category_mapping[category].dropna():
        # Create a regex pattern that matches the name within any text
        pattern = rf'.*{re.escape(name)}.*'
        replacements[pattern] = category

In [19]:
# Sources_classification_21-22_23-24
# Read the Excel file containing the category mappings
mapping_file2 = '/Users/graciaandriamiadana/Documents/Research Culture PGR project/data/Sources_classification_21-22_23-24.csv'
# category_mapping2 = pd.read_csv(mapping_file)
try:
    category_mapping2 = pd.read_csv(mapping_file2, encoding='latin1')
except UnicodeDecodeError as e:
    print(f"Error reading the file with 'latin1' encoding: {e}")
    # Try another encoding if needed
    category_mapping2 = pd.read_csv(mapping_file2, encoding='ISO-8859-1')

# Remove the unwanted characters "', " from the 'Source' column
category_mapping2['Source'] = category_mapping2['Source'].str.replace(r"[\"',]", "", regex=True)

print(category_mapping2.head(5))

                          Source Source classification
0             50% LMK Thermosafe              Internal
1                           ASDF                 Other
2  Aston Martin Formula One Team              Industry
3                     Bela - 25%              Internal
4         Belgian Rsch Inst VITO                 Other


In [20]:
# Initialize an empty dictionary for replacements
replacements2 = {}

# Iterate over each row in the category_mapping DataFrame to generate regex patterns
for index, row in category_mapping2.iterrows():
    source = row['Source']
    classification = row['Source classification']
    # Create a regex pattern that matches the source within any text
    pattern = rf'.*{re.escape(source)}.*'
    replacements2[pattern] = classification

replacements.update(replacements2)


In [21]:
replacements[r'^\?.*'] = 'not available'
replacements[r'^nan$'] = 'not available'
replacements[r'^ '] = 'not available'
replacements[r'^Belgian Rsch Inst\, VITO.*'] = 'Other'

replacements[r'.*EU \(Sage X ITN\).*'] = 'EU'
replacements[r'.*ERC.*'] = 'EU'


# Additional entries grouped under "Charities"
charities_entries = [
    'HS Barlow Charitable Trust/ Paragraf',
    'Heart Research UK',
    'Horne Family Charitable Fdn',
    'Horne Family foundation',
    'Horne Foundation',
    'Bowel Research UK',
    'Welcome',
    'Wellcome',
    'PWSA UK',
    'Prostate Cancer UK',
    'Stuart Hall Foundation (SHF).',
    'Versus arthritis',
    'ANTRUK (Antibiotic Research UK)',
    'Animal Free Research UK',
    'Barry Reed Foundation',
    "Tommy's Charity",
    'Charity',
]

# Add these entries to the replacements dictionary under "Charities"
for entry in charities_entries:
    pattern = rf'.*{re.escape(entry)}.*'
    replacements[pattern] = 'Charities'

# Additional entries grouped under "Industry"
industry_entries = [
    'AstraZeneca',
    'Artios Pharma',
    'Huawei',
    'Industrial top-up',
    'Industry',
    'Industry (Creative Assembly)',
    'LTA Cola',
    'META',
    'Meta Platforms, Inc.',
    'Microsoft',
    'COLA',
    'ICase industry partner',
    'Matching industry contribution',
    'industry'
]

# Add these entries to the replacements dictionary under "Industry"
for entry in industry_entries:
    pattern = rf'.*{re.escape(entry)}.*'
    replacements[pattern] = 'Industry'

# Additional entries grouped under "UKRI"
ukri_entries = [
    'CDA supplement',
    'AIM CDT',
    'LiDo',
    'LISS',
    'CDT',
    'EPRSC DTP CASE Conversion 2021',
]

# Add these entries to the replacements dictionary under "UKRI"
for entry in ukri_entries:
    pattern = rf'.*{re.escape(entry)}.*'
    replacements[pattern] = 'UKRI'

# Additional entries grouped under "Internal"
internal_entries = [
    'SBBS',
    'SPIR',
    'SPCS',
    'WIPH',
    'Wolfson',
    'Supervisor Project',
    'Supervisor project',
    'Faculty',
    'Faculty Match fund',
    'Faculty Match fund.  P/T rates from 01-Oct-22.',
    'Faculty match fund (50%)',
    'match funded by HSS faculty',
    'BAME studentship',
    'S&E Flexible Match funding',
    'S&E Match Funded',
    'S&E match fund (50%)',
    '50% S&E matched funding',
    'BCSC',
    'BCSC Alexandra Carrell',
    'Match Funding',
    'PHURI', # Institute at Whitechapel

]

# Add these entries to the replacements dictionary under "Internal"
for entry in internal_entries:
    pattern = rf'.*{re.escape(entry)}.*'
    replacements[pattern] = 'Internal'

# Additional entries grouped under "Other"
other_entries = [
    'DSTL',
    'Defence Science & Tech Lab',
    'Defence Science and Tech Lab, Gov UK',
    'One off payment for CDA top up back pay. Paid out on 25-Aug-23',
    'One off payment for historial 3m cost of living uplift',
    'One-off top-up => paid in wkly list 25-apr-2023 => ideally, it wasgoint to be an extension of funding for Oct-Dec 2023 => however, the supervisor, Dr Pearce, had to put this payment through now as his grant closes in June 2023',
    'S/s via private donation',
    'Government'
]

# Add these entries to the replacements dictionary under "Other"
for entry in other_entries:
    pattern = rf'.*{re.escape(entry)}.*'
    replacements[pattern] = 'Other'

# print(replacements)

df_melted['Source'] = df_melted['Source'].fillna('not available').astype(str)

replacements = {k: str(v) for k, v in replacements.items()}

for pattern, replacement in replacements.items():
    # mask = df_melted['Source'].str.contains(pattern, case=True, regex=True, na=False)
    # df_melted.loc[mask, 'Source'] = replacement
    df_melted['Source'] = df_melted['Source'].str.replace(pattern, replacement, regex=True)


In [22]:
print(len(df_melted["Source"].unique().tolist()))
sorted((df_melted["Source"].unique().tolist()))

7


['Charities', 'EU', 'Industry', 'Internal', 'Other', 'UKRI', 'not available']

In [23]:
# Group by 'Budget code' and sum the 'Amount' for each budget code
budget_totals = df_melted.groupby('Budget code')['Amount'].sum().reset_index()
budget_totals['Formatted Amount'] = budget_totals['Amount'].apply(lambda x: f'£{x:,.2f}')  # Format amount as currency with pounds (£) symbol
budget_totals.rename(columns={'Budget code': 'Budget code', 'Formatted Amount': 'Total Amount (£)'}, inplace=True)

# Display the total amounts per budget code
# print(budget_totals)

#### --- if requested by quartal: --- ####
# Apply custom date intervals to 'Date' column
df_melted['Quartal'] = df_melted['Date'].apply(get_quartal)

# Group by 'Budget code' and 'Quartal' and sum the 'Amount' for each combination
budget_quartal_totals = df_melted.groupby(['Budget code', 'Quartal'])['Amount'].sum().reset_index()

# Display the total amounts per budget code and quartal
# print(budget_quartal_totals)

# Group by 'School' and sum the 'Amount' for each School
studentship_payments_by_school = df_melted.groupby('School')['Amount'].sum().reset_index()

# Format the 'Amount' column
studentship_payments_by_school['Formatted Amount'] = studentship_payments_by_school['Amount'].apply(lambda x: f'£{x:,.2f}')  # Format amount as currency with pounds (£) symbol

# Rename columns for better display
studentship_payments_by_school.rename(columns={'School': 'School Name', 'Formatted Amount': 'Total Amount (£)'}, inplace=True)

# Display the total amounts per budget code
# print(studentship_payments_by_school)


# Group by 'Budget code' and 'Quartal' and sum the 'Amount' for each combination
studentship_payments_by_school_quartal_totals = df_melted.groupby(['School', 'Quartal'])['Amount'].sum().reset_index()

# Format the 'Amount' column
studentship_payments_by_school_quartal_totals['Formatted Amount'] = studentship_payments_by_school_quartal_totals['Amount'].apply(lambda x: f'£{x:,.2f}')  # Format amount as currency with pounds (£) symbol

# Rename columns for better display
studentship_payments_by_school.rename(columns={'School': 'School Name', 'Formatted Amount': 'Total Amount (£)'}, inplace=True)

# Display the total amounts per budget code and quartal
# print(studentship_payments_by_school_quartal_totals)

# Group by 'Source' and sum the 'Amount' for each School
studentship_payments_by_source = df_melted.groupby('Source')['Amount'].sum().reset_index()

# Format the 'Amount' column
studentship_payments_by_source['Formatted Amount'] = studentship_payments_by_source['Amount'].apply(lambda x: f'£{x:,.2f}')  # Format amount as currency with pounds (£) symbol

# Rename columns for better display
studentship_payments_by_source.rename(columns={'Source': 'Source Name', 'Formatted Amount': 'Total Amount (£)'}, inplace=True)

In [24]:
studentship_payments_by_school

,School Name,Amount,Total Amount (£)
0,BCI,1.323331e+06,"£1,323,330.56"
1,Blizard,1.032331e+06,"£1,032,330.76"
2,Dentistry,7.251658e+04,"£72,516.58"
3,EECS,2.865067e+06,"£2,865,067.12"
4,Geog,5.094137e+05,"£509,413.69"
5,History,3.050411e+05,"£305,041.06"
6,IPHS,6.172147e+05,"£617,214.73"
7,Law,4.315298e+05,"£431,529.85"
8,Maths,8.648583e+05,"£864,858.29"
9,SBBS,1.257063e+06,"£1,257,063.29"


In [25]:
# Display the total amounts per budget code
studentship_payments_by_source

,Source Name,Amount,Total Amount (£)
0,Charities,2.875590e+06,"£2,875,590.20"
1,EU,2.921631e+05,"£292,163.08"
2,Industry,4.837593e+05,"£483,759.31"
3,Internal,5.971357e+06,"£5,971,356.69"
4,Other,4.253994e+05,"£425,399.41"
5,UKRI,5.908936e+06,"£5,908,936.02"
6,not available,3.054208e+05,"£305,420.80"


In [26]:
studentship_payments_by_school.to_csv("studentship_payments_by_school_2021-22.csv")
studentship_payments_by_source.to_csv("studentship_payments_by_source_2021-22.csv")

In [27]:
# Calculate mean amount spent per source and per school
mean_amount_per_source = df_melted.groupby('Source')['Amount'].mean().to_frame(name='Mean Amount').sort_index()
mean_amount_per_school = df_melted.groupby('School')['Amount'].mean().to_frame(name='Mean Amount').sort_index()

# Calculate frequency by source and by school
frequency_by_source = df_melted['Source'].value_counts().to_frame(name='Frequency').sort_index()
frequency_by_school = df_melted['School'].value_counts().to_frame(name='Frequency').sort_index()

# Create the studentship payments by school and source tables
studentship_payments_by_school = df_melted.groupby('School').agg({'Amount': 'sum'}).sort_index()
studentship_payments_by_source = df_melted.groupby('Source').agg({'Amount': 'sum'}).sort_index()

# Save to Excel file
with pd.ExcelWriter('output2021-22.xlsx') as writer:
    mean_amount_per_source.to_excel(writer, sheet_name='Mean Amount per Source')
    frequency_by_source.to_excel(writer, sheet_name='Frequency by Source')
    studentship_payments_by_source.to_excel(writer, sheet_name='Payments by Source')
    mean_amount_per_school.to_excel(writer, sheet_name='Mean Amount per School')
    frequency_by_school.to_excel(writer, sheet_name='Frequency by School')
    studentship_payments_by_school.to_excel(writer, sheet_name='Payments by School')


In [28]:
studentship_payments_by_source
frequency_by_source
mean_amount_per_source

,Mean Amount
Source,
Charities,3309.079636
EU,4114.973005
Industry,2672.703352
Internal,1921.904308
Other,2444.824224
UKRI,1808.673406
not available,5988.643137
